In [5]:
import json
jsonl_file = "/home/ubuntu/LLaVA-Graph/evaluation/llava-13-llamav2-deplot/100-200-response.jsonl"
from datasets import load_dataset
deplot_test = load_dataset('alexshengzhili/SciCapQA-test-with-deplot', split='1_percent_as_validation[99:200]')
data = deplot_test.filter(lambda example: len(example['q_a_pairs']) > 0)
#data.remove_columns(['llava-answer'])

answers = []
with open(jsonl_file, 'r') as f:
    for line in f:
        example = json.loads(line)
        answers.append(example)


Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapQA-test-with-deplot-1ac41e44312d836e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [6]:
data[0].keys()

dict_keys(['image_file', 'id', 'caption', 'conversations', 'first_mention', 'response', 'title', 'abstract', 'q_a_pairs', 'deplot'])

In [7]:
len(data), len(answers)

(100, 100)

In [8]:
output_list = []
# for answer_dict in answers:
#     for data_dict in data:
#         if answer_dict['prompt'] == data_dict['q_a_pairs'][0][0]:
#             output_dict = data_dict.copy()
#             output_dict.update({'llava-answer': answer_dict['text']})
#             output_list.append(output_dict)
#             continue
# I can only zip the two lists together and then iterate through them
for answer_dict, data_dict in zip(answers, data):
    output_dict = data_dict.copy()
    output_dict.update({'llava-answer': answer_dict['text']})
    output_list.append(output_dict)

print(output_list)

[{'image_file': '1801.01974v2-Figure10-1.png', 'id': '1801.01974v2-Figure10-1', 'caption': 'Fig. 10. Average AUC and AUPR versus the number of synthetic ROIs generated with DSFS, 3DMM, and SHBMM according to pose and lighting effects where still-to-video FR system employs either SVM and SRC classifiers on Chokepoint (a, b) and COX-S2V (c, d) databases.', 'conversations': [{'from': 'human', 'value': '<image>\nWhat is the difference between the two graphs in Figure 10?'}, {'from': 'gpt', 'value': 'The two graphs in Figure 10 show the performance of still-to-video FR systems on the Chokepoint and COX-S2V databases, respectively. The x-axis of both graphs shows the number of synthetic ROIs generated under both pose and lighting effects, while the y-axis shows the average AUC and AUPR.'}, {'from': 'human', 'value': 'What are the key observations that can be made from the graphs?'}, {'from': 'gpt', 'value': 'The key observations that can be made from the graphs are as follows:\n\n* The perfo

In [9]:
set_image_id = set()
dedupted = []
for output_dict in output_list:
    if output_dict['image_file'] in set_image_id:
        continue
    set_image_id.add(output_dict['image_file'])
    dedupted.append(output_dict)

print(len(dedupted))

100


In [10]:
import argparse
import json
import os

import openai
import tqdm
import time
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_KEY")
system_message = """
You are a helpful and precise assistant for checking the quality of the answer.
You are given the graph's caption, the context of the graph, the abstract, tthe title

And then you are given the question, the reference answer, and the answer generated by the model. Please
think about how helpful the model answer is to the user and rate the model answer on a scale of 0 to 10, 
where 0 is not helpful at all and 10 is very helpful. Just return the floating number between 0 and 10.
"""
data = dedupted
model_name = "llava-answer"
def construct_input_string(index):
    content = dict()
    cur_example = data[index]
    content['title'] = cur_example['title']
    content['abstract'] = cur_example['abstract']
    content['caption'] = cur_example['caption']
    content['Question to the model'] = cur_example['q_a_pairs'][0][0]
    content['reference_answer'] = cur_example['q_a_pairs'][0][1]
    content['Candidate model answer'] = data[index][model_name]
    return json.dumps(content)


def get_openai_response(content_string):
    openai_response = openai.ChatCompletion.create(
                    model='gpt-4-0613',
                    messages=[{
                        'role': 'system',
                        'content': system_message
                    }, {
                        'role': 'user',
                        'content': content_string
                    }],
                    temperature=0,  # TODO: figure out which temperature is best for evaluation
                    max_tokens=500,
                )['choices'][0]['message']['content']
    return openai_response

openai_responses = []
for i in range(len(data)):
    content_string = construct_input_string(i)
    openai_response = get_openai_response(content_string)
    print(openai_response)
    openai_responses.append(openai_response)
    time.sleep(1.5)


2.0
4.5
0.0
5.5
0.0
6.5
4.5
7.5
2.0
0.0
0.0
0.0
8.5
0.0
8.5
7.5
4.5
5.0
4.5
4.5
7.0
5.0
8.5
5.0
2.0
9.5
0.0
6.5
0.0
8.5
6.5
1.0
5.0
5.5
3.0
9.5
6.5
3.5
4.5
9.5
9.5
8.5
1.0
4.0
9.5
3.0
9.5
9.5
2.0
5.0
0.0
7.0
0.0
9.5
0.0
5.5
7.5
4.0
3.0
0.0
10
2.0
7.5
4.5
5.5
8.5
0.0
4.5
2.0
6.5
2.0
2.0
2.0
9.5
8.5
7.5
8.5
8.5
7.5
5.5
2.0
8.5
0.0
2.0
2.0
2.0
0.0
0
9.5
9.5
9.5
7.5
9.5
6.0
4.5
8.5
3.0
5.5
9.5
9.5


In [11]:
len(openai_responses)

100

In [12]:
import numpy as np
openai_responses_float = [float(str) for str in openai_responses]
np.mean(openai_responses_float), np.std(openai_responses_float)

(5.01, 3.3173634109033032)

In [29]:
response_model = [item['text'] for item in answers]

In [31]:
new_data = data.add_column("llava_answer", response_model)
rated_data = new_data.add_column("openai_rating", openai_responses_float)
file_path = "/home/ubuntu/LLaVA-Graph/evaluation/llava-13b-v0/llava_answer_and_openai_rating_first_100.jsonl"
with open(file_path, 'w') as f:
    for example in rated_data:
        json_str = json.dumps(example)
        f.write(json_str + '\n')